In [218]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import chi2_contingency, fisher_exact, ttest_ind

In [6]:
f1 = 'Table 1 Data.xlsx'
f2 = 'All Data no PHI.xlsx'

In [47]:
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2, 'Demographics and Comorbidities')
df3 = pd.read_excel(f2, 'Pre PFT Data')
df4 = pd.read_excel(f2, 'Post PFT Data')
df5 = pd.read_excel(f2, 'Chest CT Scores')
cols = df5.iloc[0]
df5 = df5.iloc[1:]
df5.columns = cols
df5.reset_index(drop=True, inplace=True)
df5.drop(df5.columns[5:42], axis=1, inplace=True)

In [110]:
df5.rename(columns={'Person_ID': 'PersonID'}, inplace=True)

/var/folders/2j/cm0mpbq52bqgp7_zrf20hl3h0000gn/T/ipykernel_45624/286605221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5.rename(columns={'Person_ID': 'PersonID'}, inplace=True)


In [111]:
df5

,PersonID,COVID+ Date,Timeframe,CT Date,Months from COVID+,GGO,Consolidation,Linear/Curvilinear Band Opacities,Reticulation,Traction Bronchiectasis,...,NaN,33,Pre,Avg,0.48989898989898994,0.1464646464646465,0.04141414141414142,0.6666666666666667,0.5252525252525252,0.1414141414141414
0,9000976218,2020-06-03 00:00:00,Pre,2020-04-13 12:31:00,1.659566,0.166667,0,0,0,0,...,NaN,NaN,NaN,SD,0.803572,0.463527,0.103759,1.039164,0.921504,0.431247
1,9000282974,2020-03-17 00:00:00,Pre,2018-10-26 13:49:00,16.682443,0.5,0,0,1.5,0.833333,...,NaN,49,Post,Avg,0.513605,0.119048,0.061905,0.721088,0.659864,0.251701
2,9001674052,2020-03-20 00:00:00,Pre,2017-09-18 09:40:00,30.036073,0,0,0,0,0,...,NaN,NaN,NaN,SD,0.927859,0.399653,0.121906,1.187993,1.134456,0.602005
3,9000660522,2020-03-22 00:00:00,Pre,2016-07-19 11:36:00,44.104658,0.5,0,0,0.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9000897057,2020-03-27 00:00:00,Pre,2019-03-16 00:00:00,12.394521,2.666667,2.333333,0,2,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,9000756132,2022-01-07 00:00:00,Post,2022-03-18 09:59:00,2.315046,0.666667,0,0.166667,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,9000718049,2022-01-12 00:00:00,Post,2022-07-27 10:37:00,6.458379,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,9002664600,2022-01-14 00:00:00,Post,2022-03-19 07:21:00,2.114178,0.833333,0,0,0,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,9000416791,2022-01-20 00:00:00,Post,2022-06-08 11:36:00,4.585753,0.166667,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Hospitalized vs non-hospitalized split

In [19]:
sum(df2['Hospitalized for Infection'])
c = 0
for index, row in df2.iterrows():
    s = row['Hospitalized for Infection']
    s1 = row['Critically Ill (ICU/IMV)']
    if s == 1 or s1 == 1:
        c += 1
print(c)

79


In [50]:
hosp = df2[df2['Hospitalized for Infection'] == 1]
non_hosp = df2[df2['Hospitalized for Infection'] == 0]
hosp_precovid_data = hosp.merge(df3, on='PersonID', how='inner')
nonhosp_precovid_data = non_hosp.merge(df3, on='PersonID', how='inner')
hosp_postcovid_data = hosp.merge(df4, on='PersonID', how='inner')
nonhosp_postcovid_data = non_hosp.merge(df4, on='PersonID', how='inner')
hosp_ct_data = hosp.merge(df5, on='PersonID', how='inner')
nonhosp_ct_data = non_hosp.merge(df5, on='PersonID', how='inner')

/Users/christineyoon/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():
/Users/christineyoon/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


# Table 2

In [23]:
items = ['Months from PFT to COVID+', 'FEV1 % Pred', 'FVC % Pred', 'FEV1/FVC', 'RV/TLC', 'RV/TLC % Pred', 'TLC % Pred']

In [24]:
def avg_sd(categories, df):
    d = {}
    for cat in categories: 
        d[cat] = {}
        d[cat]['average'] = df[cat].mean()
        d[cat]['stdev'] = df[cat].std()
    return d

### Pre-covid

In [53]:
# Hospitalized 
precovid_hosp = avg_sd(items, hosp_precovid_data)
pd.DataFrame(precovid_hosp).transpose()

,average,stdev
Months from PFT to COVID+,16.120434,14.035353
FEV1 % Pred,68.100400,19.837553
FVC % Pred,68.222278,23.545983
FEV1/FVC,81.841011,10.638170
RV/TLC,48.133333,9.642708
RV/TLC % Pred,128.045455,26.792011
TLC % Pred,77.340000,16.139203


In [54]:
# Non-hospitalized 
precovid_nonhosp = avg_sd(items, nonhosp_precovid_data)
pd.DataFrame(precovid_nonhosp).transpose()

,average,stdev
Months from PFT to COVID+,13.024175,14.076038
FEV1 % Pred,74.211884,21.798640
FVC % Pred,74.530714,22.394705
FEV1/FVC,79.806842,12.461853
RV/TLC,45.354839,9.315036
RV/TLC % Pred,119.090909,25.313489
TLC % Pred,80.388889,17.419657


### Post-covid

In [55]:
# Hospitalized 
postcovid_hosp = avg_sd(items, hosp_postcovid_data)
pd.DataFrame(postcovid_hosp).transpose()

,average,stdev
Months from PFT to COVID+,8.326720,6.991263
FEV1 % Pred,66.830395,22.473869
FVC % Pred,72.100897,21.062489
FEV1/FVC,80.419495,11.081042
RV/TLC,47.026316,9.818589
RV/TLC % Pred,121.055556,20.787282
TLC % Pred,76.877551,19.026990


In [56]:
# Non-hospitalized 
postcovid_nonhosp = avg_sd(items, nonhosp_postcovid_data)
pd.DataFrame(postcovid_nonhosp).transpose()

,average,stdev
Months from PFT to COVID+,6.443491,6.107483
FEV1 % Pred,72.765714,20.157462
FVC % Pred,73.922429,20.075332
FEV1/FVC,78.860956,11.108835
RV/TLC,49.187500,11.467872
RV/TLC % Pred,122.787879,24.476721
TLC % Pred,80.047619,15.528027


### Paired t-test

In [74]:
hosp_postcovid_data['FEV1 % Pred']

0     108.0
1      77.0
2      67.0
3      98.0
4      56.0
      ...  
74     76.0
75     36.0
76     65.0
77     56.0
78     33.0
Name: FEV1 % Pred, Length: 79, dtype: float64

In [89]:
def paired_tt(pre, post, categories):
    for cat in categories: 
        # Drop NA
        pre_cleaned = pre.dropna(subset=[cat])
        post_cleaned = post.dropna(subset=[cat]) 
        # Order
        order = [val for val in list(pre_cleaned['PersonID']) if val in list(post_cleaned['PersonID'])]
        pre2 = pre_cleaned[pre_cleaned['PersonID'].isin(order)] 
        post2 = post_cleaned[post_cleaned['PersonID'].isin(order)]
        # Perform a paired t-test
        t_statistic, p_value = stats.ttest_rel(pre2[cat], post2[cat])
        # Print the results
        print(f"{cat}:")
        print(f"{len(pre) - len(pre2)} rows dropped")
        print("Paired t-test results:")
        print(f"t-statistic: {t_statistic}")
        print(f"p-value: {p_value}")

        # Determine if the result is statistically significant (e.g., at alpha = 0.05)
        alpha = 0.05
        if p_value < alpha:
            print("The difference is statistically significant (reject the null hypothesis).")
        else:
            print("There is no statistically significant difference (fail to reject the null hypothesis).")
        print("\n")

In [90]:
paired_tt(hosp_precovid_data, hosp_postcovid_data, items)

Months from PFT to COVID+:
0 rows dropped
Paired t-test results:
t-statistic: 4.849556442200632
p-value: 6.1955011220942424e-06
The difference is statistically significant (reject the null hypothesis).


FEV1 % Pred:
5 rows dropped
Paired t-test results:
t-statistic: 1.8633759508925347
p-value: 0.06643166560328993
There is no statistically significant difference (fail to reject the null hypothesis).


FVC % Pred:
1 rows dropped
Paired t-test results:
t-statistic: -1.4319774460231722
p-value: 0.15619720671115483
There is no statistically significant difference (fail to reject the null hypothesis).


FEV1/FVC:
15 rows dropped
Paired t-test results:
t-statistic: 0.13291550982184433
p-value: 0.8946838382402993
There is no statistically significant difference (fail to reject the null hypothesis).


RV/TLC:
48 rows dropped
Paired t-test results:
t-statistic: -0.46990149939033254
p-value: 0.6418233416593278
There is no statistically significant difference (fail to reject the null hypothesis).

In [216]:
paired_tt(nonhosp_precovid_data, nonhosp_postcovid_data, items)

Months from PFT to COVID+:
0 rows dropped
Paired t-test results:
t-statistic: 3.3955285030585407
p-value: 0.001139600282792014
The difference is statistically significant (reject the null hypothesis).


FEV1 % Pred:
1 rows dropped
Paired t-test results:
t-statistic: 1.3519819728031337
p-value: 0.18086087005012877
There is no statistically significant difference (fail to reject the null hypothesis).


FVC % Pred:
0 rows dropped
Paired t-test results:
t-statistic: 0.18661789968115514
p-value: 0.8525078373488731
There is no statistically significant difference (fail to reject the null hypothesis).


FEV1/FVC:
14 rows dropped
Paired t-test results:
t-statistic: -0.009373586383258366
p-value: 0.9925549877787609
There is no statistically significant difference (fail to reject the null hypothesis).


RV/TLC:
47 rows dropped
Paired t-test results:
t-statistic: -0.20220643751674694
p-value: 0.8416135485496132
There is no statistically significant difference (fail to reject the null hypothesis).

# Independent t-test

In [227]:
def ind_tt(hosp_df_pre, nonhosp_df_pre, hosp_df_post, nonhosp_df_post, categories):
    for cat in categories:
        pre_post_p = []
        for time_df in [[hosp_df_pre, nonhosp_df_pre], [hosp_df_post, nonhosp_df_post]]:
            # Separate hospitalized vs. nonhospitalized
            hosp = time_df[0][cat].dropna()
            nonhosp = time_df[1][cat].dropna()
            # Independent t-test for parameter 
            t, p = ttest_ind(hosp, nonhosp, equal_var=False) 
            pre_post_p.append(p)       
        # Output the p-values
        print(f"Pre-COVID {cat} p-value: {pre_post_p[0]}")
        print(f"Post-COVID {cat} p-value: {pre_post_p[1]}")

In [228]:
ind_tt(hosp_precovid_data, nonhosp_precovid_data, hosp_postcovid_data, nonhosp_postcovid_data, items)

Pre-COVID Months from PFT to COVID+ p-value: 0.18173378408069582
Post-COVID Months from PFT to COVID+ p-value: 0.08135790123173722
Pre-COVID FEV1 % Pred p-value: 0.08157250055812203
Post-COVID FEV1 % Pred p-value: 0.09471690079421297
Pre-COVID FVC % Pred p-value: 0.09604810024615433
Post-COVID FVC % Pred p-value: 0.5911029959601221
Pre-COVID FEV1/FVC p-value: 0.3114142918083149
Post-COVID FEV1/FVC p-value: 0.44263151877947937
Pre-COVID RV/TLC p-value: 0.21222073025944152
Post-COVID RV/TLC p-value: 0.405122776478119
Pre-COVID RV/TLC % Pred p-value: 0.13855692598159308
Post-COVID RV/TLC % Pred p-value: 0.7534624437292137
Pre-COVID TLC % Pred p-value: 0.4117706759950105
Post-COVID TLC % Pred p-value: 0.38399887644296593


# Table 3

In [125]:
hosp_ct = hosp.merge(df5, on='PersonID', how='inner')
hosp_pre_ct = hosp_ct[hosp_ct['Timeframe'] == 'Pre']
hosp_post_ct = hosp_ct[hosp_ct['Timeframe'] == 'Post']
nonhosp_ct = non_hosp.merge(df5, on='PersonID', how='inner')
nonhosp_pre_ct = nonhosp_ct[nonhosp_ct['Timeframe'] == 'Pre']
nonhosp_post_ct = nonhosp_ct[nonhosp_ct['Timeframe'] == 'Post']
conditions = ['GGO','Consolidation','Linear/Curvilinear Band Opacities','Reticulation','Traction Bronchiectasis','Honeycombing']

In [119]:
# Check
precovid_ct = df5[df5['Timeframe'] == 'Pre'] 
postcovid_ct = df5[df5['Timeframe'] == 'Post']

In [117]:
def percent_breakdown(df, categories):
    d = {}
    total = len(df)
    for cat in categories:
        d[cat] = {}
        d[cat]['0'] = len(df[df[cat] <= 0])/total*100
        d[cat]['0-1'] = len(df[(df[cat] > 0) & (df[cat] < 1)])/total*100
        d[cat]['1+'] = len(df[df[cat] >= 1])/total*100
        d[cat]['Nonzero'] = len(df[df[cat] > 0])/total*100
    return pd.DataFrame(d)

In [126]:
# Overall for hospitalized precovid patients 
percent_breakdown(hosp_pre_ct, conditions)

,GGO,Consolidation,Linear/Curvilinear Band Opacities,Reticulation,Traction Bronchiectasis,Honeycombing
0,60.000000,86.666667,86.666667,53.333333,80.0,93.333333
0-1,13.333333,0.000000,13.333333,20.000000,0.0,6.666667
1+,26.666667,13.333333,0.000000,26.666667,20.0,0.000000
Nonzero,40.000000,13.333333,13.333333,46.666667,20.0,6.666667


In [127]:
# Overall for hospitalized postcovid patients 
percent_breakdown(hosp_post_ct, conditions)

,GGO,Consolidation,Linear/Curvilinear Band Opacities,Reticulation,Traction Bronchiectasis,Honeycombing
0,50.0,87.5,70.833333,50.0,62.500000,83.333333
0-1,25.0,0.0,29.166667,12.5,8.333333,4.166667
1+,25.0,12.5,0.000000,37.5,29.166667,12.500000
Nonzero,50.0,12.5,29.166667,50.0,37.500000,16.666667


In [128]:
# Overall for nonhospitalized precovid patients 
percent_breakdown(nonhosp_pre_ct, conditions)

,GGO,Consolidation,Linear/Curvilinear Band Opacities,Reticulation,Traction Bronchiectasis,Honeycombing
0,55.555556,83.333333,83.333333,61.111111,50.000000,83.333333
0-1,27.777778,11.111111,16.666667,5.555556,22.222222,0.000000
1+,16.666667,5.555556,0.000000,33.333333,27.777778,16.666667
Nonzero,44.444444,16.666667,16.666667,38.888889,50.000000,16.666667


In [129]:
# Overall for nonhospitalized postcovid patients 
percent_breakdown(nonhosp_post_ct, conditions)

,GGO,Consolidation,Linear/Curvilinear Band Opacities,Reticulation,Traction Bronchiectasis,Honeycombing
0,64.0,88.0,80.0,56.0,60.0,76.0
0-1,28.0,12.0,20.0,28.0,20.0,12.0
1+,8.0,0.0,0.0,16.0,20.0,12.0
Nonzero,36.0,12.0,20.0,44.0,40.0,24.0


In [159]:
def chi_sq(df, categories):
    d = {}
    for cat in categories: 
        d[cat] = {}
        d[cat]['Pre'] = len(df[(df['Timeframe'] == 'Pre') & (df[cat] > 0)])
        d[cat]['Post'] = len(df[(df['Timeframe'] == 'Post') & (df[cat] > 0)])
    return pd.DataFrame(d).T
#         print(precovid)
#         print(postcovid)
#         data = pd.DataFrame({'Pre': list(precovid), 'Post': list(postcovid)})
#         chi2, p, _, _ = chi2_contingency(data)
#         print(p)

def expected_vals(d):
    total = sum(d) 
    newd = {}
    pre_total = sum(d['Pre'])
    post_total = sum(d['Post'])
    for cat in categories: 
        newd[cat] = {}
        tot = sum(d.T[cat])
        newd[cat]['Pre'] = pre_total*tot/total
        newd[cat]['Post'] = post_total*tot/total
    return pd.DataFrame(newd)

In [166]:
chi_sq(df5, conditions).T

,Pre,Post
GGO,14,21
Consolidation,5,6
Linear/Curvilinear Band Opacities,5,12
Reticulation,14,23
Traction Bronchiectasis,12,19
Honeycombing,4,10


In [214]:
def contingency(categories, df):
    d = {} 
    d['Timeframe'] = df['Timeframe']
    for cat in categories:
        d[cat] = df[cat].apply(lambda x: 1 if x > 0 else 0)     
    data = pd.DataFrame(d)
    p_values = {}
    for category in categories:
        # Creating the contingency table
        contingency_table = pd.crosstab(data[category], data['Timeframe'])
#         print(contingency_table)
        # Performing the Chi-Square Test
        chi2, p, dof, expected = chi2_contingency(contingency_table) 
        # Storing the p-value
        p_values[category] = p
    # Printing the p-values
    for category, p in p_values.items():
        print(f"Category: {category}, p-value: {p}")

In [215]:
contingency(conditions, df5)

Category: GGO, p-value: 1.0
Category: Consolidation, p-value: 0.9614389774207762
Category: Linear/Curvilinear Band Opacities, p-value: 0.45616055528041777
Category: Reticulation, p-value: 0.8598212919601241
Category: Traction Bronchiectasis, p-value: 1.0
Category: Honeycombing, p-value: 0.4972897940541593


In [211]:
contingency(conditions, nonhosp_ct)

Category: GGO, p-value: 0.8083109603381144
Category: Consolidation, p-value: 1.0
Category: Linear/Curvilinear Band Opacities, p-value: 1.0
Category: Reticulation, p-value: 0.9825611254723282
Category: Traction Bronchiectasis, p-value: 0.7337182447491325
Category: Honeycombing, p-value: 0.838963574612519
